In [1]:
from os.path import join, exists
from os import makedirs

import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from itertools import combinations, product

from brainpipe.system import study
from utils import subjects, context_su, odor_groups_wgth, odor_groups_3wgth
from similarity_funcs import compute_tps_btw, compute_tps_wth

In [21]:
"""
Compute TPSim wth ODOR BY ODOR
"""
###############################################################################
st = study('Olfacto')
path_pow = join(st.path, 'feature_new/TPSim_power_data/')
pow_file = join(path_pow, '{}_odors=all_elecs=psd_freq={}_pow_EL.npz')
path2save = join(st.path, 'feature_new/TPSim_by_odor_wth/')
savename = join(path2save, '{}_odors=all_tps=wth_elecs=psd_freq={}_{}.npz')
###############################################################################
if not exists(path2save):
    makedirs(path2save)

freqs = ['l_theta','h_theta']
dims = ['xpow_E','xpow_L']
t0, t1 = 0, 2

for su, freq in product(subjects,freqs):
    print(su)
    pow_mat = np.load(pow_file.format(su,freq),allow_pickle=True)
    time = pow_mat['time']-3
    t_sel = [i for i,t in enumerate(time) if t0<=t<t1]
    
    for dim in dims:
        step = dim.split('_')[-1]
        x = pow_mat[dim][:,:,t_sel,:]

        od_wth, tps_all_od, p_all_od, tr_list = [], [], [], []
        for od in np.unique(pow_mat['od_'+step]):
            id_trials = np.where([pow_mat['od_'+step]==od])[1]

            if len(id_trials) > 1:
                x_od = np.squeeze(x[:,:,:,id_trials])
                tps_od, p_od, tr_l = compute_tps_wth(x_od) #nelecs x ncombs
                tps_all_od.append(tps_od), p_all_od.append(p_od)
                od_wth.extend([od]*tps_od.shape[-1])
                tr_list.extend(tr_l)
        tps_all_od = np.concatenate((tps_all_od),axis=-1)
        print('tps_wth',dim,freq,tps_all_od.shape)
        p_all_od = np.concatenate((p_all_od),axis=-1)

        dico_tps = {}
        for fi in pow_mat.files:
            dico_tps[fi] = pow_mat[fi]
        dico_tps['tps'], dico_tps['pvals'] = tps_all_od, p_all_od
        dico_tps['odor'] = od_wth
        dico_tps['trials_combs'] = tr_list
        #print(freq, 'mean tps wth by elec', np.mean(tps_all_od,axis=1))
        np.savez(savename.format(su,freq,dim),**dico_tps)

-> Olfacto loaded
CHAF
[]
[]
tps_wth xpow_E l_theta (10, 2)
[]
[]
[]
[]
[]
tps_wth xpow_L l_theta (10, 7)
CHAF
[]
[]
tps_wth xpow_E h_theta (14, 2)
[]
[]
[]
[]
[]
tps_wth xpow_L h_theta (14, 7)
LEFC
[]
[]
[]
[]
[]
[]
[]
tps_wth xpow_E l_theta (31, 47)
[]
[]
[]
[]
[]
[]
[]
tps_wth xpow_L l_theta (31, 57)
LEFC
[]
[]
[]
[]
[]
[]
[]
tps_wth xpow_E h_theta (26, 47)
[]
[]
[]
[]
[]
[]
[]
tps_wth xpow_L h_theta (26, 57)
FERJ
[]
[]
[]
tps_wth xpow_E l_theta (76, 8)
[]
[]
[]
[]
[]
[]
[]
tps_wth xpow_L l_theta (76, 16)
FERJ
[]
[]
[]
tps_wth xpow_E h_theta (66, 8)
[]
[]
[]
[]
[]
[]
[]
tps_wth xpow_L h_theta (66, 16)
SEMC
[]
[]
[]
[]
[]
tps_wth xpow_E l_theta (15, 18)
[]
[]
[]
[]
[]
[]
[]
[]
tps_wth xpow_L l_theta (15, 24)
SEMC
[]
[]
[]
[]
[]
tps_wth xpow_E h_theta (15, 18)
[]
[]
[]
[]
[]
[]
[]
[]
tps_wth xpow_L h_theta (15, 24)
VACJ
[]
tps_wth xpow_E l_theta (38, 1)
[]
[]
[]
[]
[]
tps_wth xpow_L l_theta (38, 5)
VACJ
[]
tps_wth xpow_E h_theta (60, 1)
[]
[]
[]
[]
[]
tps_wth xpow_L h_theta (60, 5)
PI

In [25]:
"""
Compute TPSim BTW ODORS
"""
###############################################################################
st = study('Olfacto')
path_pow = join(st.path, 'feature_new/TPSim_power_data/')
pow_file = join(path_pow, '{}_odors=all_elecs=psd_freq={}_pow_EL.npz')
path2save = join(st.path, 'feature_new/TPSim_by_odor_btw/')
savename = join(path2save, '{}_odors=all_tps=btw_elecs=psd_freq={}_{}.npz')
###############################################################################
if not exists(path2save):
    makedirs(path2save)

freqs = ['l_theta','h_theta']
dims = ['xpow_E','xpow_L']
t0, t1 = 0, 2

for su, freq in product(subjects,freqs):
    print(su)
    pow_mat = np.load(pow_file.format(su,freq),allow_pickle=True)
    time = pow_mat['time']-3
    t_sel = [i for i,t in enumerate(time) if t0<=t<t1]
    
    for dim in dims:
        step = dim.split('_')[-1]
        x = pow_mat[dim][:,:,t_sel,:]
        
        od_btw, tps_all_od, p_all_od, combs_ods = [], [], [], []
        for od in np.unique(pow_mat['od_'+step]):
            id_trials = np.where([pow_mat['od_'+step]==od])[1]
            id_others = [i for i in range(x.shape[-1]) if i not in id_trials]
            od_target = pow_mat['od_'+step][id_trials]
            od_others = pow_mat['od_'+step][id_others]

            if len(id_trials) > 1:
                x_od = np.squeeze(x[:,:,:,id_trials])
                x_od2 = np.squeeze(x[:,:,:,id_others])
                tps_od, p_od = compute_tps_btw(x_od,x_od2) #nelecs x ncombs
                tps_all_od.append(tps_od), p_all_od.append(p_od)
                od_btw.extend([od]*tps_od.shape[-1])
                combs_ods.extend([o1+'_'+o2 for o1, o2 in product(od_target,od_others)])
        tps_all_od = np.concatenate((tps_all_od),axis=-1)
        print('tps_btw',tps_all_od.shape)
        p_all_od = np.concatenate((p_all_od),axis=-1)

        dico_tps = {}
        for fi in pow_mat.files:
            dico_tps[fi] = pow_mat[fi]
        dico_tps['tps'], dico_tps['pvals'] = tps_all_od, p_all_od
        dico_tps['odor'] = od_btw
        dico_tps['combs'] = combs_ods
        print(freq, 'combs', len(combs_ods), 'od', len(od_btw))
        np.savez(savename.format(su,freq,dim),**dico_tps)

-> Olfacto loaded
CHAF


ValueError: not enough values to unpack (expected 3, got 2)

In [28]:
"""
Compute TPSim WTH multiple Odors (by Richness, Context)
"""
###############################################################################
st = study('Olfacto')
path_pow = join(st.path, 'feature_new/TPSim_power_data/')
pow_file = join(path_pow, '{}_odors=all_elecs=psd_freq={}_pow_EL.npz')
path2save = join(st.path, 'feature_new/TPSim_by_{}_wth/')
savename = join(path2save, '{}_odors=all_tps=wth_elecs=psd_freq={}_{}.npz')
###############################################################################
cond = 'context' #'mem2', 'mem3', 'context'
freqs = ['l_theta','h_theta']
dims = ['xpow_E','xpow_L']
t0, t1 = 0,2
dict_gr = (context_su if cond == 'context' else \
            (odor_groups_wgth if cond == 'mem2' else odor_groups_3wgth))

if not exists(path2save.format(cond)):
    makedirs(path2save.format(cond))
###############################################################################

for su, freq in product(dict_gr,freqs):
    print('>> processing',su,freq,cond)
    pow_mat = np.load(pow_file.format(su,freq),allow_pickle=True)
    time = pow_mat['time']-3
    t_sel = [i for i,t in enumerate(time) if t0<=t<t1]
    
    for dim in dims:
        step = dim.split('_')[-1]
        x = pow_mat[dim][:,:,t_sel,:]

        keys_list = list(dict_gr[su].keys())
        od_lists = list(dict_gr[su].values())
        cond_wth, tps_all_od, p_all_od = [], [], []
        for num,l in enumerate(od_lists):
            id_trials = [i for i,od in enumerate(pow_mat['od_'+step]) if od in l]
            if len(id_trials) > 1:
                x_od = np.squeeze(x[:,:,:,id_trials])
                tps_od, p_od, _ = compute_tps_wth(x_od) #nelecs x ncombs
                tps_all_od.append(tps_od), p_all_od.append(p_od)
                cond_wth.extend([keys_list[num]]*tps_od.shape[-1])
        tps_all_od = np.concatenate((tps_all_od),axis=-1)
        #print('tps_wth',tps_all_od.shape,cond_wth)
        p_all_od = np.concatenate((p_all_od),axis=-1)

        dico_tps = {}
        for fi in pow_mat.files:
            dico_tps[fi] = pow_mat[fi]
        dico_tps['tps'], dico_tps['pvals'] = tps_all_od, p_all_od
        dico_tps[cond] = cond_wth
        #print(freq, 'mean tps wth by elec', np.mean(tps_all_od,axis=1))
        np.savez(savename.format(cond,su,freq,dim),**dico_tps)

-> Olfacto loaded
>> processing CHAF l_theta context
['f', 'split', 'width', 'step', 'fname', 'labels', 'channels', 'odor', 'EM_2gr', 'EM_3gr', 'xyz', 'time', 'xpow_E', 'xpow_L', 'od_E', 'od_L', 'all_odors']


KeyError: 'od_xpow_E is not a file in the archive'

In [29]:
"""
Compute TPSim BTW multiple Odors (by Richness, Context)
"""
###############################################################################
st = study('Olfacto')
path_pow = join(st.path, 'feature_new/TPSim_power_data/')
pow_file = join(path_pow, '{}_odors=all_elecs=psd_freq={}_pow_EL.npz')
path2save = join(st.path, 'feature_new/TPSim_by_{}_btw/')
savename = join(path2save, '{}_odors=all_tps=btw_elecs=psd_freq={}_{}.npz')
###############################################################################
cond = 'context' #'mem2', 'mem3', 'context'
freqs = ['l_theta','h_theta']
dims = ['xpow_E','xpow_L']
t0, t1 = 0, 2
dict_gr = (context_su if cond == 'context' else \
            (odor_groups_wgth if cond == 'mem2' else odor_groups_3wgth))

if not exists(path2save.format(cond)):
    makedirs(path2save.format(cond))
###############################################################################

for su, freq in product(dict_gr,freqs):
    print('>> processing',su,freq,cond)
    pow_mat = np.load(pow_file.format(su,freq),allow_pickle=True)
    time = pow_mat['time']-3
    t_sel = [i for i,t in enumerate(time) if t0<=t<t1]
    
    for dim in dims:
        step = dim.split('_')[-1]
        x = pow_mat[dim][:,:,t_sel,:]

        keys_list = list(dict_gr[su].keys())
        od_lists = list(dict_gr[su].values())

        cond_btw, tps_all_od, p_all_od = [], [], []
        for cond1,cond2 in combinations(range(len(keys_list)),2):
            id_trials0 = [i for i,od in enumerate(pow_mat['od_'+step])\
                                              if od in od_lists[cond1]]
            id_trials1 = [i for i,od in enumerate(pow_mat['od_'+step])\
                                              if od in od_lists[cond2]]
            x_od = np.squeeze(x[:,:,:,id_trials0])
            x_od2 = np.squeeze(x[:,:,:,id_trials1])
            tps_od, p_od = compute_tps_btw(x_od,x_od2) #nelecs x ncombs
            tps_all_od.append(tps_od), p_all_od.append(p_od)
            cond_btw.extend([keys_list[cond1]+'_'+keys_list[cond2]]*tps_od.shape[-1])

        tps_all_od = np.concatenate((tps_all_od),axis=-1)
        print('tps_btw',tps_all_od.shape)
        p_all_od = np.concatenate((p_all_od),axis=-1)

        dico_tps = {}
        for fi in pow_mat.files:
            dico_tps[fi] = pow_mat[fi]
        dico_tps['tps'], dico_tps['pvals'] = tps_all_od, p_all_od
        dico_tps[cond] = cond_btw
        #print(freq, 'mean tps wth by elec', np.mean(tps_all_od,axis=1))
        np.savez(savename.format(cond,su,freq,dim),**dico_tps)

-> Olfacto loaded
>> processing CHAF l_theta context
tps_btw (10, 24)
tps_btw (10, 28)
>> processing CHAF h_theta context
tps_btw (14, 24)
tps_btw (14, 28)
>> processing LEFC l_theta context
tps_btw (31, 133)
tps_btw (31, 168)
>> processing LEFC h_theta context
tps_btw (26, 133)
tps_btw (26, 168)
>> processing PIRJ l_theta context
tps_btw (20, 110)
tps_btw (20, 120)
>> processing PIRJ h_theta context
tps_btw (45, 110)
tps_btw (45, 120)
>> processing VACJ l_theta context
tps_btw (38, 20)
tps_btw (38, 42)
>> processing VACJ h_theta context
tps_btw (60, 20)
tps_btw (60, 42)
>> processing SEMC l_theta context
tps_btw (15, 65)
tps_btw (15, 112)
>> processing SEMC h_theta context
tps_btw (15, 65)
tps_btw (15, 112)
>> processing FERJ l_theta context
tps_btw (76, 42)
tps_btw (76, 77)
>> processing FERJ h_theta context
tps_btw (66, 42)
tps_btw (66, 77)
